In [2]:
from Bio import SeqIO
record = SeqIO.read("/home/m/genome_files/NC_014328.fasta", "fasta")
table = 11
min_pro_len = 10
count = 0
with open('/home/m/Desktop/temp/NC_014328_all_peptides10plus.faa', 'w+') as f:
    for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
        for frame in range(3):
            length = 3 * ((len(record)-frame) // 3) #Multiple of three
            for pro in nuc[frame:frame+length].translate(table).split("*"):
                if len(pro) >= min_pro_len:
                    count=count+1
                    sseq=(">count %i\t length %i\t strand %i\t frame %i\n%s\n" % (count, len(pro), strand, frame, pro))
                    f.write(sseq)

# This will generate peptides with genome positions

In [1]:
from Bio import SeqIO
def find_orfs_with_trans(seq, trans_table, min_protein_length):
    answer = []
    seq_len = len(seq)
    for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
        for frame in range(3):
            trans = str(nuc[frame:].translate(trans_table))
            trans_len = len(trans)
            aa_start = 0
            aa_end = 0
            while aa_start < trans_len:
                aa_end = trans.find("*", aa_start)
                if aa_end == -1:
                    aa_end = trans_len
                if aa_end-aa_start >= min_protein_length:
                    if strand == 1:
                        start = frame+aa_start*3
                        end = min(seq_len,frame+aa_end*3+3)
                    else:
                        start = seq_len-frame-aa_end*3-3
                        end = seq_len-frame-aa_start*3
                    answer.append((start, end, strand,
                                   trans[aa_start:aa_end]))
                aa_start = aa_end+1
    answer.sort()
    return answer

In [3]:
record = SeqIO.read("/home/m/genome_files/NC_014328.gbk","genbank")
table = 11
min_pro_len = 10
counter = 0
orf_list = find_orfs_with_trans(record.seq, table, min_pro_len)
with open('/home/m/Dropbox/collaboration_with_sam_ernesto/all_frames2faa/NC_014328_all_peptides10plus_with_positions.faa', 'w+') as f:
    for start, end, strand, pro in orf_list:
        counter =counter+1
        sseq=(">peptide_%s %s..%s length%i strand %i\n%s\n" % (counter, start, end, len(pro), strand, pro))
        f.write(sseq)